### Codio Activity 9.2: Cross-Validation with `SequentialFeatureSelector`

**Expected Time: 45 Minutes**

**Total Points: 60**

This activity uses the same example as in Video 9.3. That example uses the `shuffle` and `np.split` functions to create the train/test split indices. Then, these indices are passed to the 'cv' argument in the `SequentialFeatureSelector`. Here, rather than using a Pipeline you are to conduct the transformations by hand, and feed the resulting selected features into a `LinearRegression` estimator.


#### Index

- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)
- [Problem 3](#Problem-3)
- [Problem 4](#Problem-4)

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
from random import shuffle, seed

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error

### The Dataset

For this exercise, the built in diabetes dataset will be used.  The features are blood measurements and demographic information, and the target is a numeric measurement of diabetes progression.  The data is loaded and displayed below.  

In [16]:
diabetes = load_diabetes(as_frame = True)

In [17]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [18]:
df = diabetes.frame

In [19]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
 10  target  442 non-null    float64
dtypes: float64(11)
memory usage: 38.1 KB


[Back to top](#Index)

### Problem 1

#### Create a list of indicies and shuffle them

**10 Points**

To begin, create a list of the indices of the DataFrame `df`.  Assign this list to `all_indices`.  Then, use the `shuffle` function to shuffle the list in place.  In order to be consistent, set a `seed(42)` before calling the `shuffle` function.  

In [21]:
seed(42)
all_indices = list(range(len(df)))
shuffle(all_indices)
# (train_inds, dev_inds) = np.split(all_inds, [320])

# Answer check
all_indices[:5] #Should be [170, 226, 231, 345, 337]

[170, 226, 231, 345, 337]

[Back to top](#Index)

### Problem 2

**10 Points**

#### Split indices to train and test values

Use the `np.split` function to split the data based on the first 350 values of `all_indices`.  Assign these as arrays to `train_idx` and `test_idx` below.  

In [22]:
### GRADED
(train_idx, test_idx) = np.split(all_indices, [350])

# Answer check
print(train_idx[:5])#Should be [170 226 231 345 337]
print(test_idx[:5])#Should be [311  36 331 349 136]

[170 226 231 345 337]
[311  36 331 349 136]


[Back to top](#Index)

### Problem 3

#### Creating `SequentialFeatureSelector` object

**20 Points**

Create a `SequentialFeatureSelector` object named `selector` below that:

- Uses `LinearRegression` estimator to select features
- Select 4 features using `n_features_to_select`
- Use `train_idx` and `test_idx` in the `cv` argument
- Use `neg_mean_squared_error` as the scorer 

Use the selector to transform the data `X` given below (you'll also need `y`).  Assign the transformed data as an array to `Xt` below.

In [23]:
X = diabetes.frame.drop('target', axis = 1)
y = diabetes.frame.target

In [26]:
### GRADED
selector = SequentialFeatureSelector(
    estimator=LinearRegression(),
    n_features_to_select=4,
    cv=[[train_idx, test_idx]],
    scoring="neg_mean_squared_error",
)

Xt = selector.fit_transform(X, y)

### ANSWER CHECK
print(Xt.shape)#should have four columns
display(selector.get_feature_names_out())

(442, 4)


array(['bmi', 'bp', 's3', 's5'], dtype=object)

[Back to top](#Index)

### Problem 4

#### Using selected features in a model

**20 Points**

Now, use your features that have been selected -- `Xt` -- to build a `LinearRegression` estimator.  Name and fit your estimator as `lr`, predictions on `Xt` as `model_preds`, and determine the mean squared error assigning this as `mse` below.

In [25]:
### GRADED
lr = LinearRegression().fit(Xt, y)
model_preds = lr.predict(Xt)
mse = mean_squared_error(y, model_preds)

# Answer check
print(mse)

3015.353354928773
